<a href="https://colab.research.google.com/github/AliHassan-019/accent-detection/blob/main/notebook3328d452ea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
!pip install kagglehub

import kagglehub
mozillaorg_common_voice_path = kagglehub.dataset_download('mozillaorg/common-voice')
edolele_speech_commends_path = kagglehub.dataset_download('edolele/speech-commends')

print('Data source import complete.')


Download already complete (12931472693 bytes).
Extracting files...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.25G/2.25G [07:42<00:00, 5.23MB/s]

Extracting files...


Data source import complete.


# Importing libraries, loading and transforming data

In [7]:
!pip install -q evaluate transformers==4.28.1
!pip install -U -q datasets
!pip install -q torchaudio==0.12.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!add-apt-repository -y ppa:savoury1/ffmpeg4
!apt-get -qq install -y ffmpeg
!pip install -q mlflow

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Ignored the following yanked versions: 2.0.0
ERROR: Could not find a version that satisfies the requirement torchaudio==0.12.0+cu113 (from versions: 2.0.1, 2.0.2, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0)
ERROR: No matching distribution found for torchaudio==0.12.0+cu113
'add-apt-repository' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program

# Torch audio tutorial

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm


In [ ]:
from torchaudio.datasets import SPEECHCOMMANDS
import os


class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__(root = "/kaggle/input/", url = '.', download=False, folder_in_archive='speech-commends/')

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:

                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = load_list("validation_list.txt") + load_list("testing_list.txt")
            excludes = set(excludes)
            self._walker = [w for w in self._walker if w not in excludes]


# Create training and testing split of the data. We do not use validation in this tutorial.
train_set = SubsetSC("training")
test_set = SubsetSC("testing")

waveform, sample_rate, label, speaker_id, utterance_number = train_set[0]


In [ ]:
train_set._path

In [ ]:
print("Shape of waveform: {}".format(waveform.size()))
print("Sample rate of waveform: {}".format(sample_rate))

plt.plot(waveform.t().numpy());

In [ ]:
labels = ['no',
 'two',
 'backward',
 'four',
 'five',
 'nine',
 'right',
 'follow',
 'visual',
 'off',
 'yes',
 'six',
 'dog',
 'learn',
 'left',
 'bird',
 'forward',
 'wow',
 'zero',
 'eight',
 'bed',
 'go',
 'house',
 'tree',
 'seven',
 'on',
 'three',
 'one',
 'down',
 'stop',
 'up',
 'happy',
 'marvin',
 'cat',
 'sheila']

In [ ]:
waveform_first, *_ = train_set[0]
ipd.Audio(waveform_first.numpy(), rate=sample_rate)



In [ ]:
waveform_second, *_ = train_set[-1]
ipd.Audio(waveform_second.numpy(), rate=sample_rate)


In [ ]:
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
transformed = transform(waveform_second)

ipd.Audio(transformed.numpy(), rate=new_sample_rate)


In [ ]:
def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))


def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]


word_start = "yes"
index = label_to_index(word_start)
word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)


为了将由录音和话语组成的数据点列表转换为模型的两个批处理张量，我们实现了PyTorch DataLoader使用的collate函数，该函数允许我们分批遍历数据集。有关使用校对函数的更多信息，请参阅文档。



在整理函数中，我们还应用了重采样和文本编码。



In [ ]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)


def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets


batch_size = 256

device = 'cuda'

if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)


为了将由录音和话语组成的数据点列表转换为模型的两个批处理张量，我们实现了PyTorch DataLoader使用的collate函数，该函数允许我们分批遍历数据集。在本教程中，我们将使用卷积神经网络来处理原始音频数据。通常对音频数据进行更高级的变换，但cnn可以用来准确地处理原始数据。具体的体系结构是在本文描述的M5网络体系结构的基础上建模的。处理原始音频数据的模型的一个重要方面是其第一层滤波器的接受域。我们模型的第一个滤波器的长度是80，所以当处理以8kHz采样的音频时，接收场大约是10ms(而在4kHz时，大约是20ms)。这个大小类似于语音处理应用程序，通常使用20ms到40ms的接受域。

In [ ]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)


model = M5(n_input=transformed.shape[0], n_output=len(labels))
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10

In [ ]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [ ]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        data = transform(data)
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")

In [ ]:
log_interval = 20
n_epoch = 2

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        test(model, epoch)
        scheduler.step()

# Let's plot the training loss versus the number of iteration.
plt.plot(losses);
plt.title("training loss");


In [ ]:
def predict(tensor):
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    tensor = transform(tensor)
    tensor = model(tensor.unsqueeze(0))
    tensor = get_likely_index(tensor)
    tensor = index_to_label(tensor.squeeze())
    return tensor


waveform, sample_rate, utterance, *_ = train_set[-1]
ipd.Audio(waveform.numpy(), rate=sample_rate)

print(f"Expected: {utterance}. Predicted: {predict(waveform)}.")

In [ ]:
for i, (waveform, sample_rate, utterance, *_) in enumerate(test_set):
    output = predict(waveform)
    if output != utterance:
        ipd.Audio(waveform.numpy(), rate=sample_rate)
        print(f"Data point #{i}. Expected: {utterance}. Predicted: {output}.")
        break
else:
    print("All examples in this dataset were correctly classified!")
    print("In this case, let's just look at the last data point")
    ipd.Audio(waveform.numpy(), rate=sample_rate)
    print(f"Data point #{i}. Expected: {utterance}. Predicted: {output}.")

# Accent recognition

In [14]:
#imports
!pip install imblearn
!pip install torch
!pip install torchaudio
import pandas as pd
import gc
import re
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import torch
import torchaudio
import datasets
import transformers
print(transformers.__version__)

4.28.1


In [15]:
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

# If you already downloaded earlier, reuse that; otherwise download now
try:
    cv_root = Path(mozillaorg_common_voice_path)
except NameError:
    import kagglehub
    cv_root = Path(kagglehub.dataset_download('mozillaorg/common-voice'))

print("📁 Common Voice root:", cv_root)

def _find_one(root: Path, names):
    cands = []
    for n in names:
        cands += list(root.rglob(n))
    # prefer shortest path (usually top-level) and deterministic order
    cands = sorted(cands, key=lambda p: (len(p.parts), str(p).lower()))
    return cands[0] if cands else None

# Prefer the higher-quality validated/train split; keep test apart
train_like_names = [
    "cv-valid-train.csv","cv-valid-train.tsv",
    "validated.csv","validated.tsv",
    "train.csv","train.tsv"
]
test_like_names = [
    "cv-test.csv","cv-test.tsv","test.csv","test.tsv"
]

train_meta = _find_one(cv_root, train_like_names)
test_meta  = _find_one(cv_root, test_like_names)

if not train_meta:
    raise FileNotFoundError(
        f"Could not find any of {train_like_names} under {cv_root}. "
        "Inspect the folder to see available metadata files."
    )

def _load_meta(path: Path) -> pd.DataFrame:
    sep = "\t" if path.suffix.lower() == ".tsv" else ","
    df = pd.read_csv(path, sep=sep)
    # normalize columns that often appear in Common Voice
    df.columns = [c.strip() for c in df.columns]
    return df

dd = _load_meta(train_meta).drop_duplicates()
print(f"✅ Loaded TRAIN/VALID meta: {train_meta}  -> shape {dd.shape}")

# Keep only rows with non-null 'accent' if present (some releases may miss it)
if "accent" in dd.columns:
    dd = dd[dd["accent"].notna()].copy()
    print("🎯 Filtered non-null 'accent':", dd.shape)
else:
    print("⚠️ No 'accent' column in this release. Available columns:", list(dd.columns))

# Leave TEST set apart if available
if test_meta:
    test_df = _load_meta(test_meta).drop_duplicates()
    print(f"📦 Loaded TEST meta: {test_meta} -> shape {test_df.shape}")
else:
    test_df = None
    print("ℹ️ No explicit TEST file found; you can hold out a split from train if needed.")

# Optional: make a validation split from dd (stratify on 'accent' if present)
if "accent" in dd.columns:
    train_df, val_df = train_test_split(
        dd, test_size=0.1, random_state=42, stratify=dd["accent"]
    )
else:
    train_df, val_df = train_test_split(dd, test_size=0.1, random_state=42)

print("📊 Final splits:",
      f"\n  train: {train_df.shape}",
      f"\n  valid: {val_df.shape}",
      f"\n  test : {None if test_df is None else test_df.shape}")

# Peek
display(train_df.sample(min(5, len(train_df))) if len(train_df) else train_df.head())


📁 Common Voice root: C:\Users\MASTER\.cache\kagglehub\datasets\mozillaorg\common-voice\versions\2
✅ Loaded TRAIN/VALID meta: C:\Users\MASTER\.cache\kagglehub\datasets\mozillaorg\common-voice\versions\2\cv-valid-train.csv  -> shape (195776, 8)
🎯 Filtered non-null 'accent': (64711, 8)
ℹ️ No explicit TEST file found; you can hold out a split from train if needed.
📊 Final splits: 
  train: (58239, 8) 
  valid: (6472, 8) 
  test : None


,filename,text,up_votes,down_votes,age,gender,accent,duration
84436,cv-valid-train/sample-084436.mp3,i need you to be spontaneous he asked me out t...,2,1,thirties,female,us,NaN
156304,cv-valid-train/sample-156304.mp3,the alchemist dismounted slowly and the boy di...,3,0,twenties,female,indian,NaN
150834,cv-valid-train/sample-150834.mp3,never advertise razors by shaving a monkey it'...,1,0,twenties,male,us,NaN
123801,cv-valid-train/sample-123801.mp3,i'm going to become bitter and distrustful of ...,3,0,fourties,male,us,NaN
42956,cv-valid-train/sample-042956.mp3,that doesn't happen to just anyone,1,0,twenties,male,us,NaN


In [99]:
dd = pd.read_csv(
    r"C:\Users\MASTER\.cache\kagglehub\datasets\mozillaorg\common-voice\versions\2\cv-valid-train.csv"
).drop_duplicates()
dd = dd[dd['accent'].notna()]
print(dd.shape)
dd.sample(5)


(64711, 8)


,filename,text,up_votes,down_votes,age,gender,accent,duration
95467,cv-valid-train/sample-095467.mp3,i thought you were gone,2,0,fourties,male,us,NaN
87032,cv-valid-train/sample-087032.mp3,plastic surgery has become more popular,1,0,twenties,male,australia,NaN
163899,cv-valid-train/sample-163899.mp3,and he immediately felt peace in his heart,3,0,teens,male,england,NaN
39647,cv-valid-train/sample-039647.mp3,she doesn't know it yet,1,0,fifties,female,indian,NaN
87298,cv-valid-train/sample-087298.mp3,one day the earth began to tremble and the nil...,1,0,thirties,male,canada,NaN


In [16]:
dd['accent'].value_counts()

accent
us                30997
england           14938
indian             4490
australia          4287
canada             3901
scotland           1556
african            1173
newzealand         1153
ireland             944
philippines         326
wales               262
bermuda             196
malaysia            182
singapore           124
hongkong             99
southatlandtic       83
Name: count, dtype: int64

In [17]:
from collections import Counter
labels = [lang for lang, _ in Counter(dd['accent']).most_common(5)]
labels

['us', 'england', 'indian', 'australia', 'canada']

In [18]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 40000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

print(id2label, '\n\n', label2id)

{0: 'us', 1: 'england', 2: 'indian', 3: 'australia', 4: 'canada'} 

 {'us': 0, 'england': 1, 'indian': 2, 'australia': 3, 'canada': 4}


# Load and preprocess data

In [19]:
dd = dd[dd['accent'].isin(labels)]
dd['label'] = dd['accent'].apply(lambda x: label2id[x])
dd = dd[['filename', 'label']]
print(dd.shape)
dd.sample(5)

(58613, 2)


,filename,label
23351,cv-valid-train/sample-023351.mp3,0
143117,cv-valid-train/sample-143117.mp3,3
178567,cv-valid-train/sample-178567.mp3,4
195766,cv-valid-train/sample-195766.mp3,1
159266,cv-valid-train/sample-159266.mp3,0


In [20]:
# random undersampling of all but minority class
rus = RandomUnderSampler(random_state=83, sampling_strategy='not minority')
y = dd[['label']]
dd = dd.drop(['label'], axis=1)
dd, y_resampled = rus.fit_resample(dd, y)
del y
dd['label'] = y_resampled
del y_resampled
gc.collect()

986

In [10]:
dd['label'].value_counts()

label
0    30997
1    14938
2     4490
3     4287
4     3901
Name: count, dtype: int64

In [103]:
from pathlib import Path
import pandas as pd
import numpy as np

# 1) Build an index of all audio files under cv_root (mp3 + wav just in case)
audio_paths = list(cv_root.rglob("*.mp3")) + list(cv_root.rglob("*.wav"))
print(f"Found {len(audio_paths)} audio files under {cv_root}")

# 2) Map basename -> full path (lowercased for safety)
index = {p.name.lower(): str(p.resolve()) for p in audio_paths}

# 3) Add a 'basename' column derived from metadata's filename
dd['basename'] = dd['filename'].apply(lambda s: Path(str(s)).name.lower())

# 4) Map to absolute paths via the index
dd['abs_path'] = dd['basename'].map(index)

# 5) Report missing and drop them
missing = dd['abs_path'].isna().sum()
print(f"Resolved: {len(dd) - missing}/{len(dd)} | Missing: {missing}")

# (Optional) peek a few missing basenames to understand gaps
if missing:
    print("Missing examples:", dd.loc[dd['abs_path'].isna(), 'basename'].head(5).tolist())

# Keep only rows we can actually read
dd = dd.dropna(subset=['abs_path']).copy()


Found 380368 audio files under C:\Users\MASTER\.cache\kagglehub\datasets\mozillaorg\common-voice\versions\2
Resolved: 64711/64711 | Missing: 0


In [105]:
from pathlib import Path
import os, numpy as np, torch, torchaudio

# --- 1) Point pydub to your FFmpeg binaries (no PATH fiddling needed) ---
from pydub import AudioSegment
AudioSegment.converter = r"C:\Program Files\FFmpeg\bin\ffmpeg.exe"
AudioSegment.ffprobe   = r"C:\Program Files\FFmpeg\bin\ffprobe.exe"

# (Optional) also add to PATH for other libs that rely on PATH lookups
ffbin = r"C:\Program Files\FFmpeg\bin"
if ffbin not in os.environ.get("PATH",""):
    os.environ["PATH"] = os.environ["PATH"] + ";" + ffbin

def _resample_torch(wav: torch.Tensor, orig_sr: int, target_sr: int) -> torch.Tensor:
    if orig_sr == target_sr:
        return wav
    return torchaudio.transforms.Resample(orig_sr, target_sr)(wav)

def _resample_numpy(x: np.ndarray, orig_sr: int, target_sr: int) -> np.ndarray:
    try:
        from scipy.signal import resample_poly
        up, down = target_sr, orig_sr
        return np.stack([resample_poly(ch, up, down) for ch in x], axis=0)
    except Exception:
        ratio = target_sr / orig_sr
        t_new = np.arange(int(x.shape[1] * ratio))
        t_old = np.linspace(0, x.shape[1]-1, x.shape[1])
        return np.stack([np.interp(t_new, t_old, ch) for ch in x], axis=0).astype(np.float32)

def load_audio_any(path: str, target_sr: int = 16000, mono: bool = True):
    """Try torchaudio first; fallback to pydub+FFmpeg."""
    # --- torchaudio first ---
    try:
        wav, sr = torchaudio.load(path)
        if not torch.is_floating_point(wav):
            wav = wav.float() / (1 << 31)
        if mono and wav.shape[0] > 1:
            wav = wav.mean(dim=0, keepdim=True)
        wav = _resample_torch(wav, sr, target_sr)
        return wav.contiguous(), target_sr
    except Exception:
        pass

    # --- fallback: pydub (uses the FFmpeg paths set above) ---
    seg = AudioSegment.from_file(path)   # auto-detect via extension
    sr = seg.frame_rate
    ch = seg.channels
    sw = seg.sample_width  # bytes/sample

    samples = np.array(seg.get_array_of_samples())
    if ch > 1:
        samples = samples.reshape((-1, ch)).T  # [C, T]
    else:
        samples = samples[None, :]            # [1, T]

    max_val = float(1 << (8*sw - 1))          # normalize to [-1, 1]
    samples = (samples.astype(np.float32) / max_val)

    if sr != target_sr:
        samples = _resample_numpy(samples, sr, target_sr)

    wav = torch.from_numpy(samples.astype(np.float32))
    if mono and wav.shape[0] > 1:
        wav = wav.mean(dim=0, keepdim=True)
    return wav.contiguous(), target_sr

# --- 2) Use the resolved local path we built earlier ---
file_path = dd['abs_path'].iloc[0]
print("Loading:", file_path, "| exists:", Path(file_path).exists())

audio, rate = load_audio_any(file_path, target_sr=16000, mono=True)
print("✅ Loaded")
print("   Shape:", tuple(audio.shape), "| SR:", rate)


Loading: C:\Users\MASTER\.cache\kagglehub\datasets\mozillaorg\common-voice\versions\2\cv-valid-train\cv-valid-train\sample-000005.mp3 | exists: True
✅ Loaded
   Shape: (1, 93312) | SR: 16000


In [106]:
# audio, rate already = (waveform, RATE_HZ) from load_audio_any
audio = audio.numpy().reshape(-1)


In [108]:
# Ensure mono [T] NumPy array regardless of how audio is shaped
if isinstance(audio, torch.Tensor):
    if audio.ndim == 2 and audio.shape[0] == 1:   # [1, T]
        audio_np = audio.squeeze(0).numpy()
    elif audio.ndim == 2 and audio.shape[0] > 1:  # [C, T] -> average to mono
        audio_np = audio.mean(dim=0).numpy()
    else:  # already [T]
        audio_np = audio.numpy()
else:
    # already NumPy
    audio_np = audio

print("Final shape:", audio_np.shape)


Final shape: (93312,)


In [109]:
from tqdm import tqdm

def get_transform_audio(file):
    # use the resolved absolute path from dd
    file_path = dd.loc[file, 'abs_path'] if isinstance(file, (int, np.integer)) else file
    if Path(str(file_path)).exists():
        audio, rate = load_audio_any(file_path, target_sr=RATE_HZ, mono=True)
    else:
        raise FileNotFoundError(f"File not found: {file_path}")

    # Ensure 1-D numpy waveform
    if isinstance(audio, torch.Tensor):
        if audio.ndim == 2 and audio.shape[0] > 1:  # stereo -> mono
            audio = audio.mean(dim=0)
        else:
            audio = audio.squeeze()
        audio_np = audio.numpy()
    else:
        audio_np = np.array(audio, dtype=np.float32)

    # Truncate/pad to fixed MAX_LENGTH
    audio_np = audio_np[:MAX_LENGTH]
    return audio_np

# Apply to all rows with progress bar
tqdm.pandas()
dd['audio'] = dd['abs_path'].progress_apply(get_transform_audio)
gc.collect()


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64711/64711 [2:38:42<00:00,  6.80it/s]


1154

In [85]:
%%time
dd = dd.drop(['filename'], axis=1)
gc.collect()

CPU times: total: 250 ms
Wall time: 242 ms


4257

In [86]:
dd.sample(5)

,label,abs_path
132405,1,C:\Users\MASTER\.cache\kagglehub\datasets\mozi...
103549,3,C:\Users\MASTER\.cache\kagglehub\datasets\mozi...
192911,3,C:\Users\MASTER\.cache\kagglehub\datasets\mozi...
91054,1,C:\Users\MASTER\.cache\kagglehub\datasets\mozi...
60879,4,C:\Users\MASTER\.cache\kagglehub\datasets\mozi...


In [87]:
from datasets import Dataset
dd = Dataset.from_pandas(dd)
gc.collect()

8

In [88]:
from collections import Counter
Counter(dd['label']).items()

dict_items([(0, 3901), (1, 3901), (2, 3901), (3, 3901), (4, 3901)])

In [89]:
dd = dd.train_test_split(test_size=0.25)
gc.collect()
dd

DatasetDict({
    train: Dataset({
        features: ['label', 'abs_path', '__index_level_0__'],
        num_rows: 14628
    })
    test: Dataset({
        features: ['label', 'abs_path', '__index_level_0__'],
        num_rows: 4877
    })
})

# Load facebook/wav2vec2-base-960h model

In [116]:
import os
from transformers import (
    AutoProcessor, AutoFeatureExtractor,
    Wav2Vec2Config, Wav2Vec2ForSequenceClassification
)

# -------- Local-friendly settings --------
MODEL_NAME = "facebook/wav2vec2-base-960h"   # base acoustic model
# Use a stable local cache folder (adjust if you like)
os.environ.setdefault("HF_HOME", r"C:\Users\MASTER\.cache\huggingface")
os.environ.setdefault("TRANSFORMERS_CACHE", r"C:\Users\MASTER\.cache\huggingface\hub")

# label maps you already created earlier
# labels, label2id, id2label must exist
num_labels = len(labels)

# -------- Processor / feature extractor --------
# Prefer AutoProcessor (new API), fall back to AutoFeatureExtractor if needed
try:
    processor = AutoProcessor.from_pretrained(MODEL_NAME)
except Exception:
    processor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

# -------- Config tailored to accent classification --------
config = Wav2Vec2Config.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    finetuning_task="accent-classification",
    problem_type="single_label_classification"
)

# -------- Model with classification head --------
# ignore_mismatched_sizes=True lets us reuse the base encoder
# even though the classifier head shape is new.
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config,
    ignore_mismatched_sizes=True
)

# (Optional) freeze the base to warm up the classifier first
FREEZE_BASE = False  # set to True to train only the classifier layer first
if FREEZE_BASE:
    for p in model.wav2vec2.parameters():
        p.requires_grad = False

# Count trainable params (in millions)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6
print(f"Trainable params: {trainable:.3f} M")



tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForSequenceClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be 

Trainable params: 94.570 M


In [122]:
def preprocess_function(batch):
    inputs = feature_extractor(batch['audio'], sampling_rate=RATE_HZ, max_length=MAX_LENGTH, truncation=True)
    inputs['input_values'] = inputs['input_values'][0]
    return inputs

dd['train'] = dd['train'].map(preprocess_function, remove_columns="audio", batched=False)
gc.collect()
dd['test'] = dd['test'].map(preprocess_function, remove_columns="audio", batched=False)
gc.collect()

TypeError: preprocess_function() got an unexpected keyword argument 'remove_columns'

In [112]:
import evaluate

accuracy = evaluate.load("accuracy")

from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    # Compute the ROC AUC score
    predictions = eval_pred.predictions
    predictions = np.exp(predictions)/np.exp(predictions).sum(axis=1, keepdims=True)
    label_ids = eval_pred.label_ids
    roc_auc = roc_auc_score(label_ids, predictions, average='macro', multi_class='ovr')

    # Calculate accuracy using the loaded accuracy metric
    acc_score = accuracy.compute(predictions=predictions.argmax(axis=1), references=label_ids)['accuracy']

    return {
        "roc_auc": roc_auc,
        "accuracy": acc_score
    }

# Training and validation

In [113]:
from transformers import TrainingArguments, Trainer
batch_size=8
warmup_steps=50
weight_decay=0.02
num_train_epochs=10
model_name = "english_accents_classification"
training_args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-5, # 3e-5
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=1,
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    save_strategy='epoch',
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd["train"],
    eval_dataset=dd["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

KeyError: 'train'

In [114]:
trainer.evaluate()

ValueError: You should supply an instance of `transformers.BatchFeature` or list of `transformers.BatchFeature` to this method that includes input_values, but you provided ['label']

In [115]:
trainer.train()

ValueError: You should supply an instance of `transformers.BatchFeature` or list of `transformers.BatchFeature` to this method that includes input_values, but you provided ['label']

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

pipe=pipeline('audio-classification',model=model_name,device=0)

In [ ]:
# us example
audio,rate=torchaudio.load('/kaggle/input/common-voice/cv-valid-test/cv-valid-test/sample-000003.mp3')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

In [ ]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)

In [97]:
# england example
audio,rate=torchaudio.load('/kaggle/input/common-voice/cv-valid-test/cv-valid-test/sample-000008.mp3')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

RuntimeError: Couldn't find appropriate backend to handle uri /kaggle/input/common-voice/cv-valid-test/cv-valid-test/sample-000008.mp3 and format None.

In [98]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)

NameError: name 'audio' is not defined

In [ ]:
# indian example
audio,rate=torchaudio.load('/kaggle/input/common-voice/cv-valid-test/cv-valid-test/sample-000033.mp3')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

In [ ]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)

In [ ]:
# australia example
audio,rate=torchaudio.load('/kaggle/input/common-voice/cv-valid-test/cv-valid-test/sample-000065.mp3')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

In [ ]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)

In [ ]:
# canada example
audio,rate=torchaudio.load('/kaggle/input/common-voice/cv-valid-test/cv-valid-test/sample-000037.mp3')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio)

In [ ]:
from IPython.display import Audio
Audio(audio,rate=RATE_HZ)